In [38]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My\ Drive/Colab Notebooks/OursRepository/public-opinion-monitor
# 
# !pip install thulac
# !pip install jieba

In [1]:
from HANModel import *
import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer
import numpy
import time, math
import torch.utils.data as data
import os
import pandas as pd
import json
import thulac
thulac = thulac.thulac()
import jieba

Model loaded succeed


In [2]:
# import torch.functional as F
# 
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([[0, 1,2,4,5],[1, 4,3,2,9]])
print(embedding(input))
embedding2 = nn.Embedding(10, 3, padding_idx=2)
# input = torch.LongTensor([[1,2,0,5, 6,7,8,9]])
print(embedding2(input))
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding3 = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([1])
print(embedding3(input))
input = torch.LongTensor([0])
print(embedding3(input))



tensor([[[-0.5285, -0.8154, -0.1730],
         [ 1.1920,  0.5945,  0.3039],
         [ 0.0969,  0.9607,  0.0874],
         [-0.6594,  1.2062, -0.6054],
         [-0.9235, -0.7378, -1.2172]],

        [[ 1.1920,  0.5945,  0.3039],
         [-0.6594,  1.2062, -0.6054],
         [-0.6973, -0.2179,  0.1628],
         [ 0.0969,  0.9607,  0.0874],
         [-1.0904,  0.2862,  0.1861]]], grad_fn=<EmbeddingBackward>)
tensor([[[ 1.0822,  0.6495, -0.0792],
         [ 2.4958,  1.7958,  0.2695],
         [ 0.0000,  0.0000,  0.0000],
         [-0.3829,  0.0271, -0.0781],
         [-0.3386, -0.4504,  0.8223]],

        [[ 2.4958,  1.7958,  0.2695],
         [-0.3829,  0.0271, -0.0781],
         [-1.3836,  1.1781,  2.2553],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.2275, -0.1155,  0.3388]]], grad_fn=<EmbeddingBackward>)
tensor([[4.0000, 5.1000, 6.3000]])
tensor([[1.0000, 2.3000, 3.0000]])


## 读取词向量
建立词语列表

In [6]:
from gensim.models import KeyedVectors, Word2Vec

# file = '../../PretrainedData/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'
# file = '../../DataSets/Word2Vect/xingrong_50_thulac/word2vect_50_w5.model'
# file = '../../DataSets/Word2Vect/xiejunjie_300_jieba/wiki_han_word2vec_300维度.model'
file = '../../DataSets/Word2Vect/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding_Min.txt'
# word2vec = Word2Vec.load(file)
word2vec = KeyedVectors.load_word2vec_format(file, binary=False,limit=100000)
# word2vec = KeyedVectors.load_word2vec_format(file, binary=False)
word2vec.init_sims(replace=True)  # 神奇，很省内存，可以运算most_similar
word2vec.vector_size

200

In [7]:
print(word2vec)
# print(word2vec.wv.vocab)
# print(len(word2vec.index2word))
print(len(word2vec.wv.index2word))
print(word2vec.wv.index2word[0])
print(word2vec.wv.index2word[1])
print(word2vec.wv.index2word[2])
print(word2vec.wv.index2word[1522])
print(word2vec.wv.index2entity[1522])
print(word2vec.similar_by_word('中国'))
print(word2vec.similar_by_word('天才'))
print(word2vec.wv)
print('word2vec.wv.vocab ---- >', word2vec.wv.vocab)
print(word2vec.wv.index2word)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: Deprecation

In [9]:
wordEmbedding = [word2vec.wv[word]  for word in word2vec.wv.index2word]
word2index = { word:i for i, word in enumerate(word2vec.wv.index2word)}
print(wordEmbedding[:2])
print(word2index['中国'])
print(word2index['天才'])


C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[array([ 0.0991139 ,  0.10946613, -0.09485413, -0.08118325,  0.03383048,
        0.07484313,  0.02328013,  0.00525041, -0.08925699,  0.05493119,
       -0.1078811 ,  0.03095761, -0.0186241 , -0.02372592, -0.08212436,
       -0.04660979,  0.06389652,  0.07494219,  0.07355529,  0.03957621,
        0.03823885, -0.09416068, -0.10144191,  0.12308748,  0.09416068,
        0.07652722, -0.01495872,  0.09946062, -0.01223445,  0.01817831,
       -0.00371491, -0.07390202,  0.03249311, -0.03313703, -0.09475507,
        0.11773799,  0.10847548, -0.02714363, -0.00559714,  0.01263071,
       -0.09009904, -0.09926249,  0.11278479,  0.00158503, -0.06394605,
       -0.07533845, -0.0761805 ,  0.04200329,  0.00500275,  0.03298843,
       -0.10436431, -0.09430927, -0.00321959,  0.02833241,  0.06315354,
       -0.07850851,  0.10104565,  0.0229334 ,  0.02773802, -0.01505778,
        0.07395155, -0.05666482, -0.05195926,  0.05344522, -0.01867363,
        0.02550908,  0.04467803, -0.10124378, -0.04913592,  0.0

## 读取训练数据

In [10]:
def compute_ngrams(word, num_min = 1, num_max = 3):
    ngrams =[]
    for ngram_length in range(num_min, min(len(word), num_max) + 1):
        for i in range(len(word) - ngram_length + 1):
            # print(i, i + ngram_length)
            ngrams.append(word[i : i + ngram_length])
    # print(ngrams)
    return list(set(ngrams))

print(compute_ngrams('you'))
print(compute_ngrams('I Think'))
print(compute_ngrams('中华人民共和国万岁'))

['ou', 'y', 'o', 'you', 'yo', 'u']
['h', 'I ', 'Th', 'Thi', 'hin', 'I', 'in', 'k', 'nk', 'n', 'T', 'I T', ' Th', 'ink', 'i', ' T', ' ', 'hi']
['民共', '华人', '华', '人民', '和国', '和', '和国万', '万岁', '人', '国万', '万', '中华', '国万岁', '民共和', '共', '华人民', '人民共', '中华人', '国', '民', '中', '岁', '共和国', '共和']


In [11]:
# 从词向量文本文件 word2vec 中获取词向量，如果获取到直接返回，若没有获取到，那么把这个词拆开
# 成为 ngrams 的新词组，并在 word2vec 中找新词组中的词向量并相加取平均，最后得到平均词向量输出
def wordVec(word, word2vec, min_n = 1, max_n = 3):
    # 确认词向量维度
    word_size = word2vec.wv.syn0[0].shape[0]

    # 如果在词典之中，直接返回词向量
    if word in word2vec.wv.vocab.keys():
        return word2vec[word]
    else:
        # 计算word的ngrams词组
        ngrams = compute_ngrams(word, min_n, max_n)
        # 不在词典的情况下
        word_vec = numpy.zeros(word_size, dtype=numpy.float32)
        ngrams_found = 0
        ngrams_single = [ng for ng in ngrams if len(ng) == 1]
        ngrams_more = [ng for ng in ngrams if len(ng) > 1]
        # 先只接受2个单词长度以上的词向量
        for ngram in ngrams_more:
            if ngram in word2vec.wv.vocab.keys():
                word_vec += word2vec[ngram]
                ngrams_found += 1
                #print(ngram)
        # 如果，没有匹配到，那么最后是考虑单个词向量
        if ngrams_found == 0:
            for ngram in ngrams_single:
                word_vec += word2vec[ngram]
                ngrams_found += 1
        if word_vec.any():
            return word_vec / max(1, ngrams_found)
        else:
            # 不抛出异常，而是打印提示，并返回0向量。
            print(KeyError('all ngrams for word %s absent from model' % word))
            return word_vec

In [12]:
class DotDict(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self
        

In [13]:
import pandas as pd

dic = {'a':[1, 2, 3, 4], 'b':[5, 6, 7, 8],
'c':[9, 10, 11, 12], 'd':[13, 14, 15, 16]}
df1=pd.DataFrame(dic)
print(df1)
df2=df1.sample(frac=0.75)
print(df2)
# rowlist=[]
# for indexs in df2.index:
#     rowlist.append(indexs)
df3=df1.drop(df2.index.to_list(),axis=0)
print(df3)

   a  b   c   d
0  1  5   9  13
1  2  6  10  14
2  3  7  11  15
3  4  8  12  16
   a  b   c   d
2  3  7  11  15
1  2  6  10  14
0  1  5   9  13
   a  b   c   d
3  4  8  12  16


In [14]:
def isNan(a):
    return a != a

class RatingData(data.Dataset):
    def __init__(self, path, 
                 word2index, 
                 max_row = -1, 
                 trainTestRate = 0.85, 
                 isTrain = True, 
                 wordCuter = thulac,
                 clean_file_name = 'ratings_clean_4_HAN.csv',
                 ):
        self.token_list = []
        self.label_list = []
        # self.token_positions = torch.tensor([i for i in range(100)])

        print(' balance_data.csv 所在path:',path) # 地址不应该包含 ratings.csv

        ratings_clean_filename = os.path.join(path, clean_file_name)
        ratings_filename = os.path.join(path, 'balance_data_means.csv')
        if os.path.isfile(ratings_clean_filename):
            clean_pd = pd.read_csv(ratings_clean_filename)
        else:
            print('没有找到缓存的文件%s, 读取源文件%s'%(ratings_clean_filename, ratings_filename))
            ratings_pd = pd.read_csv(ratings_filename)
            print('开始生成缓存文件%s'%(ratings_clean_filename))
            clean_pd = pd.DataFrame({
                'labels':[],
                'data':[],
            })
            nonRatingCount = 0
            for i, row in ratings_pd.iterrows():
                if max_row != -1 and i > max_row:
                    break
                if not isinstance(row['data'], str) or row['data'] == '':
                    # print(i + 1, row['comment'])
                    nonRatingCount += 1
                    continue
                r0 = row['labels']
                if r0 == -1:
                    r0 = 2
                    
                if i % 10000 == 9999:
                    print(i + 1, r0)

                words = list(wordCuter.cut(row['data']))
                # 0 在词向量集中是‘，’，换个词向量集可能表示其他
                token = [ word2index[words[i]] if i < len(words) and words[i] in word2index else 0 
                          for i in range(100)] 
                
                newRow = DotDict()

                newRow.labels = [r0]
                newRow.data = [json.dumps(token)]

                clean_pd = clean_pd.append(pd.DataFrame(newRow), ignore_index=True)
            print('空的评论数量： %d'%(nonRatingCount))
            clean_pd.to_csv(ratings_clean_filename)

        # 读取
        if isTrain:
            temp_pd = clean_pd.sample(frac=trainTestRate)
        else:
            temp_pd = clean_pd.sample(frac=trainTestRate)
            temp_pd = clean_pd.drop(temp_pd.index.tolist(), axis=0)

        for i, row in temp_pd.iterrows():
            if max_row != -1 and i > max_row:
                break

            self.label_list.append(torch.tensor(row['labels']).long())
            self.token_list.append(torch.from_numpy(numpy.array( json.loads(row['data']) ) ).long())

    def __getitem__(self, index):
        return self.token_list[index], self.label_list[index]#, self.token_positions

    def __len__(self):
        return len(self.label_list)
    

In [15]:
ratingData = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             # max_row= 200000,
                             isTrain=True,
                             trainTestRate = 0.8,
                             wordCuter= jieba,
                             clean_file_name='balance_data_4_HAN_means.csv',
                             )
trainLoader = torch.utils.data.DataLoader(dataset=ratingData,
                                          batch_size=128,
                                          shuffle = True,
                                          num_workers = 8,
                                          )
ratingData2 = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             # max_row= 200000,
                             isTrain=False,
                             trainTestRate = 0.8,
                             wordCuter= jieba,
                             clean_file_name='balance_data_4_HAN_means.csv',
                             )
testLoader = torch.utils.data.DataLoader(dataset=ratingData2,
                                          batch_size=128,
                                          shuffle = True,
                                          num_workers = 8,
                                          )
print(len(ratingData.label_list))
print(len(ratingData2.label_list))


 balance_data.csv 所在path: ../../DataSets/yf_dianping
没有找到缓存的文件../../DataSets/yf_dianping\balance_data_4_HAN_means.csv, 读取源文件../../DataSets/yf_dianping\balance_data_means.csv
开始生成缓存文件../../DataSets/yf_dianping\balance_data_4_HAN_means.csv
10000 2
20000 2
30000 2
40000 2
50000 2
60000 2
70000 2
80000 2
90000 2
100000 2
110000 2
120000 2
130000 2
140000 2
150000 2
160000 2
170000 2
180000 2
190000 2
200000 2
210000 2
220000 2
230000 2
240000 2
250000 2
260000 2
270000 2
280000 2
290000 2
300000 2
320000 2
330000 2
340000 2
350000 2
360000 2
370000 2
380000 2
390000 2
400000 2
410000 2
420000 2
430000 2
440000 2
450000 2
460000 2
470000 2
480000 2
490000 2
500000 2
510000 2
520000 2
530000 2
540000 2
550000 2
560000 2
570000 2
580000 2
590000 2
600000 2
610000 2
620000 2
630000 2
640000 2
650000 2
660000 2
690000 2
700000 2
710000 2
720000 2
730000 2
740000 2
750000 2
760000 2
770000 2
780000 2
800000 2
810000 2
820000 2
830000 2
840000 2
850000 2
860000 2
870000 2
880000 2
890000 2
900000

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\fuliu\AppData\Local\Temp\jieba.cache
Loading model cost 1.518 seconds.
Prefix dict has been built successfully.


In [16]:
#创建模型
wordEmbedding = torch.FloatTensor(wordEmbedding)
num_embeddings = len(word2vec.wv.index2word)
model = HAN(num_embeddings, 
            num_classes = 3,
            embedding_dim=word2vec.wv.vector_size, 
            num_words = 100,
            hidden_size_gru = 300,
            hidden_size_att = 600,
            )
print(model)

modelParams = model.parameters()
for param in modelParams:
    if len(param.data.shape) > 1:
        print('---', param.data.shape, param.data)
        torch.nn.init.kaiming_normal(param.data)
        print('--->', param.data)
        
model.embed.from_pretrained(wordEmbedding)


C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:17: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


HAN(
  (embed): Embedding(99999, 200, padding_idx=0)
  (GRU1): GRU(200, 300, batch_first=True, bidirectional=True)
  (self_attention1): SelfAttention(
    (W): Linear(in_features=600, out_features=600, bias=True)
    (U): Linear(in_features=600, out_features=1, bias=True)
  )
  (GRU2): GRU(600, 300, batch_first=True, bidirectional=True)
  (self_attention2): SelfAttention(
    (W): Linear(in_features=600, out_features=600, bias=True)
    (U): Linear(in_features=600, out_features=1, bias=True)
  )
  (fc): Linear(in_features=600, out_features=3, bias=True)
)
--- torch.Size([99999, 200]) tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.3421,  0.6240,  0.7641,  ...,  0.1275,  0.0196, -0.3185],
        [ 1.4047,  0.1457,  0.0804,  ...,  0.4322, -0.7157, -0.5866],
        ...,
        [ 0.1949, -0.3447, -0.3022,  ..., -0.1634,  0.4749, -0.8786],
        [-0.1259,  0.0478,  1.8187,  ...,  0.9310,  1.2421, -0.5992],
        [-0.6094, -0.7478,  2.0046,  ..., -0.3

Embedding(99999, 200)

## 开始训练过程

In [ ]:
def trainOneEpoch(epoch, model:HAN, trainLoader, optimizer:Optimizer, lossFunc):
    if torch.cuda.is_available():
        model = model.cuda()
        lossFunc = lossFunc.cuda()

    model.train()

    startTime = time.time()
    for i, (x, y) in enumerate(trainLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        loss = lossFunc(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 99:
            print('Epoch %d, %d/%d, loss:%f ' % (epoch, i, len(trainLoader), loss))
        # if i > 2000:
        #     break 
    print('Epoch %d cost time: %.3fs' % (epoch, time.time() - startTime))


def testModel(epoch, model:HAN, testLoader):
    if torch.cuda.is_available():
        model = model.cuda()

    model.eval()

    total = 0
    correct = 0

    startTime = time.time()
    for i, (x, y) in enumerate(testLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)

        total += len(y)
        correct += predicted.data.eq(y.data).cpu().sum().numpy()
        

        if i % 50 == 49:
            print('Epoch Test %d, %d/%d, \npredicted:%s, \ntarget:%s' % (epoch, i, len(testLoader), 
                                                                     str(predicted),
                                                                     str(y)))
        # if i > 2000:
        #     break 
    print('Epoch Test %d cost time: %.3fs' % (epoch, time.time() - startTime))
    print('准确率： %.3f' % (correct / total))
    return correct / total


def train(nepoch, model, modelSavePath, isLoad, lr=0.0002):
    last_acc = 0
    if isLoad: 
      model.load_state_dict(torch.load(modelSavePath))
      last_acc = testModel(epoch, model, testLoader)
    # optimizer=torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.001)
    optimizer=torch.optim.Adam(model.parameters(), lr=0.0002)
    lossFunc =torch.nn.CrossEntropyLoss()
    for epoch in range(nepoch):
        trainOneEpoch(epoch, model, trainLoader, optimizer, lossFunc)
        acc = testModel(epoch, model, testLoader)
        if last_acc < acc:
          torch.save(model.state_dict(), modelSavePath)
        last_acc = acc
    



In [ ]:

train(30, model, 'EmotionAnalyzeModelData_300_600.model', True, lr=0.0002)

In [ ]:

def eval(model, modelSavePath, isLoad = True):
    if isLoad: model.load_state_dict(torch.load(modelSavePath))
    testModel(0, model, testLoader)

eval(model, 'EmotionAnalyzeModelData_300_600.model', isLoad = True)

In [ ]:
lossFunc =torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
_, predicted = torch.max(input.data, 1)
output = lossFunc(input, target)
print('input',input, '\n target',target, '\n output', output)
output = lossFunc(input, predicted)
print('predicted',predicted, '\n target',target, '\n output', output)

print(predicted.data.eq(target).cpu().sum())
print(target.data.eq(predicted).cpu().sum())

In [ ]:
# 何凯明初始化

w = torch.Tensor(3, 5, 2)
print(w)
print(nn.init.kaiming_uniform(w))
print(w)
w = torch.Tensor(3, 5, 2)
print(w)
print(torch.nn.init.kaiming_normal(w))
print(w)